In [27]:
# Imports needed for the script
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont

from sklearn.tree import DecisionTreeClassifier

import pandas as pd
import numpy as np

In [94]:
from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[0, 7.5, 0, 3], iris=True, legend=False, plot_training=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap, linewidth=10)
    if not iris:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    if plot_training:
        plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris-Setosa")#yellow
        plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris-Versicolor")#blue
        plt.plot(X[:, 0][y==2], X[:, 1][y==2], "g^", label="Iris-Virginica") #green
        plt.axis(axes)
    if iris:
        plt.xlabel("Petal length", fontsize=14)
        plt.ylabel("Petal width", fontsize=14)
    else:
        plt.xlabel(r"$x_1$", fontsize=18)
        plt.ylabel(r"$x_2$", fontsize=18, rotation=0)
    if legend:
        plt.legend(loc="lower right", fontsize=14)

In [82]:
df = pd.read_csv('TrainingSet.csv', index_col = 0 )
names=['URLCount','R1Time', 'CrawlIssues', 'WebsitePriority', 'ErrorCount', 'R2Time', 'IncorrectGuidance']
print(df.shape)
df

(132, 41)


,DateReceived,Website,URLCount,R1Time,CrawlIssues,WebReviewDif,WebsitePriority,ErrorCount,R2Time,TeamLeadSBQ,...,IncorrectGuidance,Other?,Was it a PSF?,MXQA_Encoded,TeamLead_Encoded,QM_Encoded,Month_Encoded,DP_Encoded,DualReviewer_Encoded,Batch Status
632,10/18/2021,deerstags.com,17,140,100,5,QM Check Urgent,0,71,0,...,False,0,False,8.0,2.0,6.0,3.0,2.0,21.0,0
621,10/18/2021,mgchemicals.com,194,500,0,4,QM Check Needed,1,190,0,...,False,0,False,34.0,4.0,6.0,3.0,7.0,NaN,0
703,10/26/2021,godogfun.com,30,90,0,1,PSF,0,30,0,...,False,0,True,26.0,3.0,7.0,3.0,8.0,NaN,1
223,11/16/2021,tourit.com,20,120,0,1,PSF,4,70,1,...,False,0,True,29.0,3.0,6.0,2.0,8.0,NaN,1
14,NaT,manicpanic.com,40,139,25,2,None,3,71,0,...,False,0,False,40.0,3.0,1.0,0.0,2.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,10/7/2021,purplepandastore.com,25,120,0,5,QM Check Needed,0,60,0,...,False,0,False,23.0,5.0,6.0,3.0,8.0,NaN,1
913,9/27/2021,logitech.com,151,139,0,2,PSF,2,71,0,...,False,0,True,23.0,5.0,6.0,4.0,8.0,NaN,1
791,9/27/2021,rollplay.com,20,139,100,2,PSF,2,71,0,...,False,0,True,32.0,2.0,8.0,4.0,8.0,NaN,1
29,NaT,aleradetails.com,40,139,25,2,None,3,71,0,...,False,0,False,17.0,6.0,3.0,1.0,6.0,NaN,0


In [ ]:
df.groupby('CrawlIssues').groups
print(df.groupby('DP_Encoded').groups)
df.groupby('DualReviewer_Encoded').groups
df.groupby('TeamLead_Encoded').groups
df.groupby('Batch Status').groups

In [23]:
cv = KFold(n_splits=10) # Numero deseado de "folds" que haremos
accuracies = list()
max_attributes = len(list(df))
depth_range = range(1, max_attributes + 1)

# Testearemos la profundidad de 1 a cantidad de atributos +1
#divido mis datos para entrenar y de prueba, usualmente se hace un 70/30
f_train = df[:round(len(df)*0.7)] 
f_valid = df[round(len(df)*0.7):]

In [92]:
#X = f_train['URLCount','R1Time', 'CrawlIssues', 'WebsitePriority', 'ErrorCount', 'R2Time', 'IncorrectGuidance']
features = ['URLCount','R1Time', 'CrawlIssues', 'ErrorCount', 'R2Time', 'IncorrectGuidance']

x = df.loc[:,features]
y = df.loc[:, 'Batch Status']

#Transform_To_array
x_array = x.to_numpy()
y_array = y.to_numpy()
x_array

array([[17, 140, 100, 0, 71, False],
       [194, 500, 0, 1, 190, False],
       [30, 90, 0, 0, 30, False],
       [20, 120, 0, 4, 70, False],
       [40, 139, 25, 3, 71, False],
       [40, 139, 25, 3, 71, False],
       [10, 139, 0, 2, 71, False],
       [48, 139, 100, 2, 71, False],
       [30, 120, 0, 5, 90, False],
       [38, 150, 100, 0, 60, False],
       [30, 120, 0, 9, 80, False],
       [40, 139, 25, 3, 71, False],
       [22, 80, 0, 1, 40, False],
       [40, 139, 25, 3, 71, False],
       [84, 139, 100, 2, 71, False],
       [22, 139, 0, 2, 71, False],
       [46, 200, 0, 0, 15, False],
       [28, 120, 0, 0, 40, True],
       [40, 139, 25, 3, 71, False],
       [40, 139, 25, 3, 71, False],
       [40, 140, 0, 3, 100, False],
       [40, 139, 25, 3, 71, False],
       [28, 90, 0, 0, 80, False],
       [24, 150, 100, 0, 40, False],
       [30, 120, 100, 0, 50, False],
       [40, 139, 25, 3, 71, False],
       [22, 120, 100, 0, 90, False],
       [40, 139, 25, 3, 71, False]

In [93]:
tree_clf = DecisionTreeClassifier(max_depth=8, random_state=42)
tree_clf.fit(x_array, y_array)

DecisionTreeClassifier(max_depth=8, random_state=42)